# Physics-Informed Neural Networks (PINNs) for Radio Frequency Signal Analysis

## 1. Introduction

Physics-Informed Neural Networks (PINNs) are a class of neural networks that integrate **physical laws** directly into the learning process. Unlike standard neural networks, which rely solely on data, PINNs enforce **governing equations** (e.g., differential equations) as part of the loss function. This allows the model to:

- Respect physical constraints.
- Reduce the need for large labelled datasets.
- Generalise better in regions where data is sparse or noisy.

In the context of radio frequency (RF) signal analysis, PINNs can model wave propagation, interference, and electromagnetic interactions while leveraging prior knowledge from Maxwell's equations or simplified wave equations.

---

## 2. Core Concept

A PINN approximates the solution \(u(x,t)\) of a PDE by minimising a **composite loss function**:

\[
\mathcal{L} = \mathcal{L}_{\text{data}} + \lambda \mathcal{L}_{\text{physics}}
\]

Where:

- \(\mathcal{L}_{\text{data}} = \frac{1}{N} \sum_i \left| u_\text{pred}(x_i, t_i) - u_\text{obs}(x_i, t_i) \right|^2\) is the standard supervised loss.
- \(\mathcal{L}_{\text{physics}} = \frac{1}{M} \sum_j \left| \mathcal{N}[u_\text{pred}](x_j, t_j) \right|^2\) is the PDE residual loss.
- \(\mathcal{N}[u]\) represents the differential operator of the governing PDE (e.g., wave equation, Maxwell’s equations).

The network is trained to **simultaneously fit the data and satisfy the physics** at collocation points.

---

## 3. Why PINNs for RF Signal Analysis

RF signals propagate according to well-known physics (Maxwell's equations or their simplified forms). Challenges in RF signal modelling include:

- Sparse or noisy measurements from antennas.
- Complex boundary conditions (urban, indoor, or satellite environments).
- Nonlinear effects in materials and media.

PINNs offer:

1. **Data efficiency:** Physical laws guide the network, reducing required labelled samples.
2. **Generalisation:** Predicts fields in regions with no direct measurements.
3. **Flexibility:** Can handle time-dependent, multi-dimensional, or coupled RF phenomena.
4. **Integration of prior knowledge:** PDEs describing wave propagation, dispersion, or interference can be enforced explicitly.

---

## 4. Example: 1D Wave Propagation

Consider a 1D RF wave \(u(x,t)\) governed by:

\[
\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}
\]

- **Inputs:** Spatial coordinate \(x\), time \(t`  
- **Output:** Field amplitude \(u(x,t)\)  
- **Data loss:** Observed RF waveform samples at specific \((x,t)\)  
- **Physics loss:** Residual of the wave equation at collocation points  

The PINN learns a function \(u_\theta(x,t)\) that fits the measurements while obeying the wave equation, enabling accurate interpolation and extrapolation of RF fields.

---

## 5. Diagram: RF Propagation and PINN Residuals

      RF Source
         |
         v
-------------------  <- Antenna/Environment
| |
| Field u(x,t) |
| +----------------+----------------+
| | Collocation points for PDE |
| | residual enforcement |
| +----------------+----------------+
| |
-------------------
^
|
PINN predicts u(x,t)
and enforces PDE residuals

- **Blue points:** Observed RF data used for supervised loss.
- **Red points:** Collocation points where PDE residual is enforced.
- The network outputs \(u_\theta(x,t)\) that satisfies both **data fit** and **physics**.

---

## 6. Extensions to Real-World RF Scenarios

- **2D/3D propagation:** Model RF coverage in buildings, urban areas, or satellite footprints.  
- **Nonlinear effects:** Incorporate material properties or interference.  
- **Inverse problems:** Infer source positions or environmental parameters from observed signals.  
- **Time-series analysis:** Combine with recurrent architectures for dynamic signals.

---

## 7. Summary

PINNs bridge the gap between **data-driven learning** and **physics-based modelling**. In RF signal analysis, they enable:

- Accurate modelling with limited data.
- Enforcement of wave physics in complex environments.
- Improved predictive capability in both observed and unobserved regions.

By integrating PINNs into RF workflows, engineers and scientists can reduce reliance on dense measurements while maintaining physically consistent predictions.


In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd

In [ ]:
# -----------------
# PINN definition
# -----------------
# ## Class: PINN
# Defines a fully connected neural network used as the physics-informed neural network (PINN).
# - Input: (x, t)
# - Output: u(x,t), the predicted RF field amplitude
# - Architecture: feedforward MLP with tanh activations
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers)-1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
        self.activation = nn.Tanh()
    
    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            x = self.activation(layer(x))
        return self.layers[-1](x)

In [ ]:
# -----------------
# PDE residual
# -----------------
# ## Function: wave_residual
# Computes the residual of the 1D wave equation:
#   u_tt - c^2 u_xx = 0
# where:
#   - u = model(x, t)
#   - u_tt = second derivative wrt time
#   - u_xx = second derivative wrt space
#
# Inputs:
#   model: PINN instance
#   x: spatial coordinate tensor
#   t: temporal coordinate tensor
#   c: wave propagation speed
# Output:
#   residual tensor
def wave_residual(model, x, t, c=1.0):
    # Concatenate inputs
    xt = torch.cat([x, t], dim=1).requires_grad_(True)
    u = model(xt)
    
    # First derivatives
    u_t = autograd.grad(u, xt, torch.ones_like(u), retain_graph=True, create_graph=True)[0][:,1:2]
    u_x = autograd.grad(u, xt, torch.ones_like(u), retain_graph=True, create_graph=True)[0][:,0:1]

    # Second derivatives
    u_tt = autograd.grad(u_t, xt, torch.ones_like(u_t), retain_graph=True, create_graph=True)[0][:,1:2]
    u_xx = autograd.grad(u_x, xt, torch.ones_like(u_x), retain_graph=True, create_graph=True)[0][:,0:1]

    # PDE residual: u_tt - c^2 u_xx = 0
    return u_tt - (c**2) * u_xx


In [ ]:
# -----------------
# Training example
# -----------------
# ## Main block
# Example training script:
# - Defines a PINN with architecture [2, 50, 50, 50, 1]
# - Creates synthetic RF signal data (sine wave)
# - Samples collocation points for enforcing PDE residual
# - Trains with Adam optimiser using combined data + physics loss

if __name__ == "__main__":
    # Network: input (x,t) → output u(x,t)
    model = PINN([2, 50, 50, 50, 1])
    
    # Optimiser
    optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Example training data (synthetic RF samples)
    N_data = 100
    x_data = torch.rand((N_data,1)) * 2 - 1  # domain [-1,1]
    t_data = torch.rand((N_data,1)) * 1      # domain [0,1]
    u_data = torch.sin(2*torch.pi*(x_data - t_data))  # synthetic RF waveform
    
    xt_data = torch.cat([x_data, t_data], dim=1)

    # Collocation points for PDE residual
    N_f = 1000
    x_f = torch.rand((N_f,1)) * 2 - 1
    t_f = torch.rand((N_f,1)) * 1

    # Training loop
    for epoch in range(2000):
        optimiser.zero_grad()
        
        # Data loss
        u_pred = model(xt_data)
        loss_data = torch.mean((u_pred - u_data)**2)
        
        # Physics loss
        f_pred = wave_residual(model, x_f, t_f, c=1.0)
        loss_phys = torch.mean(f_pred**2)
        
        # Total loss
        loss = loss_data + loss_phys
        loss.backward()
        optimiser.step()
        
        if epoch % 200 == 0:
            print(f"Epoch {epoch}: Loss={loss.item():.6f} Data={loss_data.item():.6f} Phys={loss_phys.item():.6f}")